In [3]:
!wget https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/reviews.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!wget https://raw.githubusercontent.com/soni-ratnesh/Sentiment-Analysis-RNN/master/data/labels.txt 

'wget' is not recognized as an internal or external command,
operable program or batch file.


# Import Dependencies

In [5]:
from string import punctuation
from collections import Counter
import numpy as np
import pandas as pd
import random

# Load in the Data

In [6]:
# Read data from text files
with open("reviews.txt") as f:
  reviews = f.read()

with open("labels.txt") as f:
  labels = f.read()

# Data Exploration

In [7]:
reviews[: 1000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

In [8]:
# Characters present in reviews 
set(reviews)

{'\n',
 ' ',
 '.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [9]:
labels[: 1000]

'positive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\

# Data Preprocessing

In [10]:
# Convert alphabets to lowercase
text = reviews.lower()

# Remove the punctuations
text = ''.join([char  for char in text  if char not in punctuation])

# Remove '\n' character
reviews_split = text.split('\n') # List of reviews
text = ' '.join(reviews_split)

text[: 5000]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t    story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent m

In [11]:
labels = labels.split()
labels[: 10]

['positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative']

In [12]:
# Check if zero-length reviews are present
review_lengths = Counter(len(review)  for review in reviews_split)
print(f"Zero-length reviews: {review_lengths[0]}")

Zero-length reviews: 1


In [13]:
# Get index of reviews having non-zero length
non_zero_idxs = [idx  for idx, review in enumerate(reviews_split)  if len(review)!=0]

# Remove zero-length reviews and their corresponding labels
reviews_split = [reviews_split[idx]  for idx in non_zero_idxs]
labels = [labels[idx]  for idx in non_zero_idxs]

In [14]:
print(f"No. of reviews: {len(reviews_split)}")
print(f"No. of labels : {len(labels)}")

No. of reviews: 25000
No. of labels : 25000


In [15]:
# Get a list of words used in reviews
words = text.split()
print(words[: 15])
print(f"\nThere are {len(words)} words in reviews.")

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other']

There are 6020196 words in reviews.


In [16]:
# Get vocabulary
vocabulary = set(words)
print(f"There are {len(vocabulary)} words in the vocabulary.")

There are 74072 words in the vocabulary.


### Create Dictionary


The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

    Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers start at 1, not 0. Also, convert the reviews to integers and store the reviews in a new list called reviews_ints.


In [17]:
# Encode the words to integers
# The integers start from 1 (and not 0) as we'll pad our input vectors with zeros later
vocab_to_int = {word: idx  for idx, word in enumerate(vocabulary, 1)}


dict(list(vocab_to_int.items())[:10])

{'honostly': 1,
 'hallier': 2,
 'hamburglar': 3,
 'teachers': 4,
 'robber': 5,
 'inactivity': 6,
 'reeking': 7,
 'jay': 8,
 'downplaying': 9,
 'suxz': 10}

In [18]:
# int_vocab = dict(enumerate(unique_words))

# dict(list(int_vocab.items())[:10])

### Tokenize Reviews

In [19]:
reviews_int = [] # Will store tokenized reviews

# Tokenize each review
for review in reviews_split:
  reviews_int.append([vocab_to_int[word]  for word in review.split()])

In [20]:
rand_idx = random.randint(0, len(reviews_split))

print(f"REVIEW:\n   {reviews_split[rand_idx]}\n")
print(f"   No. of words in the review: {len(reviews_split[rand_idx].split())}\n")

print("-" * 20)

print(f"\nTOKENIZED VERSION:\n   {reviews_int[rand_idx]}\n")
print(f"   No. of tokens in the tokenized version: {len(reviews_int[rand_idx])}")


REVIEW:
   recently shown on cable tv the movie opens with a disclaimer distancing itself from any co  operation of real life persons that in itself is an eye catcher  yet the script and acting from the main characters is superb and i found myself engrossed throughout  due in no small way to the crisp  thoughtful and interesting dialogue  the film is about a meeting on one day between two real life musical  legends  who formerly composed together then seperated  the film captures the essence of their lives and philosophies  in a story which proffers an explanation for their initial  split   what is so impressive is that the actors give such seemingly realistic portrayals of the characters they play  faults and all  that this viewer at least was left believing i was witnessing a true event in almost every detail  the great skill of this play is that with astute writing and fine acting a movie basically about  two of us  talking can make an excellent picture  worthy of at least an  out o

### Encode Labels

In [21]:
encoded_labels = np.array([1 if label=="positive" else 0  for label in labels])
print(f"No. of labels: {len(encoded_labels)}")
print(encoded_labels[:20])

No. of labels: 25000
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]


In [22]:
# Get length of the longest review
max_len = max(len(review)  for review in reviews_int)

# Get the average length of reviews
avg_len = sum([len(review)  for review in reviews_int]) / len(reviews_int)

print(f"The length of the longest review is {max_len}.")
print(f"The average length of reviews is {avg_len:.2f}")

The length of the longest review is 2514.
The average length of reviews is 240.81


In [23]:
def pad_reviews_with_zeros(reviews_int, seq_len):
  features = np.zeros((len(reviews_int), seq_len), dtype=np.int32)

  for idx, review in enumerate(reviews_int):
    features[idx][-len(review):] = np.array(review)[:seq_len] 

  return features

In [24]:
features = pad_reviews_with_zeros(reviews_int, seq_len=512)
features

array([[    0,     0,     0, ..., 64387, 11507, 43404],
       [    0,     0,     0, ..., 67069,  4817, 44175],
       [    0,     0,     0, ..., 43753, 67461, 67383],
       ...,
       [    0,     0,     0, ...,  6764, 67433,  6321],
       [    0,     0,     0, ...,  3181,   307, 58039],
       [    0,     0,     0, ...,  6609, 68126, 22189]])

In [25]:
type(encoded_labels)

numpy.ndarray

In [26]:
features.shape, encoded_labels.shape

((25000, 512), (25000,))

In [27]:
import tensorflow as tf

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels,
                                                    test_size=0.2,
                                                    shuffle=True)


print(f"""
X_train shape: {X_train.shape}
y_train shape: {y_train.shape}
X_test shape : {X_test.shape}
y_test shape : {y_test.shape}""")


X_train shape: (20000, 512)
y_train shape: (20000,)
X_test shape : (5000, 512)
y_test shape : (5000,)


In [30]:
one, zero = 1, 0

for  i in y_train[:100]:
  if i == 1:
    one = one+1
  else:
    zero = zero+1

print(one,zero)

44 57


In [31]:
y_test[:20]

array([1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0])

### Generate Datasets

In [32]:
def get_training_dataset(X_train, y_train, batch_size=32):
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  train_dataset = train_dataset.shuffle(buffer_size=len(X_train), reshuffle_each_iteration=True)
  train_dataset = train_dataset.batch(batch_size)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

  return train_dataset

In [33]:
def get_validation_dataset(X_test, y_test, batch_size=32):
  validation_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  validation_dataset = validation_dataset.batch(batch_size)

  return validation_dataset

In [34]:
# Get training dataset
train_dataset = get_training_dataset(X_train, y_train, batch_size=32)

# Get validaton dataset
validation_dataset = get_validation_dataset(X_test, y_test, batch_size=32)

In [35]:
# Get sample training dataset
sample_x, sample_y = train_dataset.as_numpy_iterator().next()

print(f'Input shape: {sample_x.shape}') 
print(f'Input: \n{sample_x}')
print("\n")
print(f'Label shape: {sample_y.shape}') 
print(f'Label: \n{sample_y}')

Input shape: (32, 512)
Input: 
[[    0     0     0 ... 61043 40745 64387]
 [    0     0     0 ...  4090  6609 55188]
 [    0     0     0 ... 30072 44946 42363]
 ...
 [    0     0     0 ...  7486 10625 45155]
 [    0     0     0 ... 50770 64387 47885]
 [    0     0     0 ... 69667 61038 21008]]


Label shape: (32,)
Label: 
[1 0 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 1]


In [43]:
# from tensorflow.keras.layers import Embedding
# embedding = Embedding()


# # MAX_VOCAB_LENGTH = 10000 # max number of words to have in our vocabulary
# # MAX_LENGTH = 15 # max length our sequences will be

# embedding = tf.keras.layers.Embedding(input_dim=len(vocab_int)+1, # adding 1 to fit Keras embedding (requirement)
#                                       output_dim=512, 
#                                       embeddings_initializer="uniform",
#                                       input_length=2046) # length of the padded sequence


from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=(len(vocab_to_int)),
                             output_dim=50,
                             embeddings_initializer="uniform",
                             input_length=2046)

In [44]:
embedding(features[0])

<tf.Tensor: shape=(512, 50), dtype=float32, numpy=
array([[-0.00213534,  0.04891384, -0.00414396, ...,  0.04368666,
         0.04260508, -0.02575293],
       [-0.00213534,  0.04891384, -0.00414396, ...,  0.04368666,
         0.04260508, -0.02575293],
       [-0.00213534,  0.04891384, -0.00414396, ...,  0.04368666,
         0.04260508, -0.02575293],
       ...,
       [-0.03488366,  0.01446364,  0.04339284, ..., -0.04247081,
         0.04138663, -0.0171083 ],
       [-0.0154518 , -0.02630123,  0.04614926, ...,  0.04892632,
        -0.02031949,  0.02675143],
       [ 0.04913988,  0.0027313 ,  0.02383209, ..., -0.00965338,
         0.02303442, -0.01759497]], dtype=float32)>

In [36]:
# rev = [[12, 13, 15],
#        [45, 85, 64, 45, 63, 78, 45],
#        [16, 75, 96, 42]]